# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)
lat_lngs = zip(lats, lngs)
lat_lngs

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs


In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [4]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
 
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

10

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [5]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [6]:
# Assemble the OpenWeatherMap's API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [7]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kodiak
Processing Record 2 of Set 1 | hobart
Processing Record 3 of Set 1 | colac
Processing Record 4 of Set 1 | vaini
Processing Record 5 of Set 1 | arraial do cabo
Processing Record 6 of Set 1 | ambovombe
Processing Record 7 of Set 1 | port augusta
Processing Record 8 of Set 1 | chuy
Processing Record 9 of Set 1 | mahebourg
Processing Record 10 of Set 1 | padang
-----------------------------
Data Retrieval Complete      
-----------------------------


In [8]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

10

## Add the weather data to a new DataFrame.

In [9]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kodiak,57.7900,-152.4072,45.91,53,0,18.41,US,2022-10-25 02:11:56
1,Hobart,-42.8794,147.3294,60.66,96,100,6.91,AU,2022-10-25 02:16:47
2,Colac,-38.3333,143.5833,68.16,89,99,4.97,AU,2022-10-25 02:16:47
3,Vaini,-21.2000,-175.2000,77.16,57,20,12.66,TO,2022-10-25 02:16:47
4,Arraial Do Cabo,-22.9661,-42.0278,71.62,86,100,6.80,BR,2022-10-25 02:16:48
5,Ambovombe,-25.1667,46.0833,68.27,91,57,3.15,MG,2022-10-25 02:16:48
6,Port Augusta,-32.5000,137.7667,70.83,47,15,7.61,AU,2022-10-25 02:16:48
7,Chuy,-33.6971,-53.4616,55.40,93,2,6.33,UY,2022-10-25 02:16:49
8,Mahebourg,-20.4081,57.7000,71.26,83,75,3.44,MU,2022-10-25 02:16:49
9,Padang,-0.9492,100.3543,75.25,81,100,1.41,ID,2022-10-25 02:16:49


In [10]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [11]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity','Cloudiness','Wind Speed']

# Recreate the DataFrame by using the new column order
city_data_df = city_data_df[new_column_order]

# Display sample data
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Kodiak,US,2022-10-25 02:11:56,57.7900,-152.4072,45.91,53,0,18.41
1,Hobart,AU,2022-10-25 02:16:47,-42.8794,147.3294,60.66,96,100,6.91
2,Colac,AU,2022-10-25 02:16:47,-38.3333,143.5833,68.16,89,99,4.97
3,Vaini,TO,2022-10-25 02:16:47,-21.2000,-175.2000,77.16,57,20,12.66
4,Arraial Do Cabo,BR,2022-10-25 02:16:48,-22.9661,-42.0278,71.62,86,100,6.80
5,Ambovombe,MG,2022-10-25 02:16:48,-25.1667,46.0833,68.27,91,57,3.15
6,Port Augusta,AU,2022-10-25 02:16:48,-32.5000,137.7667,70.83,47,15,7.61
7,Chuy,UY,2022-10-25 02:16:49,-33.6971,-53.4616,55.40,93,2,6.33
8,Mahebourg,MU,2022-10-25 02:16:49,-20.4081,57.7000,71.26,83,75,3.44
9,Padang,ID,2022-10-25 02:16:49,-0.9492,100.3543,75.25,81,100,1.41


In [12]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [14]:
# Set the output file name
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")